In [1]:
import numpy as np
import pandas as pd
from itertools import product

class FuzzySet:
    """Represents a fuzzy set with membership values"""
    def __init__(self, name, elements, membership_values):
        self.name = name
        self.elements = np.array(elements)
        self.membership = np.array(membership_values)
    
    def __repr__(self):
        return f"FuzzySet({self.name}): {dict(zip(self.elements, self.membership))}"

class FuzzyRelation:
    """Represents a fuzzy relation (2D fuzzy set)"""
    def __init__(self, name, set1, set2, relation_matrix):
        self.name = name
        self.set1 = set1
        self.set2 = set2
        self.relation_matrix = np.array(relation_matrix)
    
    def __repr__(self):
        df = pd.DataFrame(self.relation_matrix, 
                         index=self.set1.elements, 
                         columns=self.set2.elements)
        return f"{self.name}:\n{df}"

def cartesian_product(fuzzy_set1, fuzzy_set2):
    """Create a fuzzy relation from Cartesian product of two fuzzy sets"""
    matrix = np.zeros((len(fuzzy_set1.elements), len(fuzzy_set2.elements)))
    
    for i, elem1 in enumerate(fuzzy_set1.elements):
        for j, elem2 in enumerate(fuzzy_set2.elements):
            # Min operation for Cartesian product
            matrix[i][j] = min(fuzzy_set1.membership[i], fuzzy_set2.membership[j])
    
    relation_name = f"{fuzzy_set1.name} × {fuzzy_set2.name}"
    return FuzzyRelation(relation_name, fuzzy_set1, fuzzy_set2, matrix)

def max_min_composition(relation1, relation2):
    """
    Perform max-min composition of two fuzzy relations.
    relation1: R(X×Y), relation2: S(Y×Z)
    Result: R∘S(X×Z)
    """
    if relation1.set2.elements.size != relation2.set1.elements.size:
        raise ValueError("Incompatible relations: set2 of R1 must match set1 of R2")
    
    n = relation1.relation_matrix.shape[0]  # |X|
    m = relation2.relation_matrix.shape[1]  # |Z|
    result_matrix = np.zeros((n, m))
    
    for i in range(n):
        for j in range(m):
            # Max-min: max over all y of min(R[x,y], S[y,z])
            min_values = np.minimum(relation1.relation_matrix[i, :], 
                                   relation2.relation_matrix[:, j])
            result_matrix[i][j] = np.max(min_values)
    
    result = FuzzyRelation(f"{relation1.name}∘{relation2.name}", 
                          relation1.set1, relation2.set2, result_matrix)
    return result

# Example Usage
print("=== Creating Fuzzy Sets ===")
A = FuzzySet("A", ['a1', 'a2', 'a3'], [0.6, 0.8, 0.5])
B = FuzzySet("B", ['b1', 'b2'], [0.7, 0.9])
C = FuzzySet("C", ['c1', 'c2', 'c3'], [0.5, 0.8, 0.6])

print(f"{A}\n{B}\n{C}\n")

print("=== Cartesian Products ===")
R = cartesian_product(A, B)
S = cartesian_product(B, C)
print(f"{R}\n")
print(f"{S}\n")

print("=== Max-Min Composition ===")
result = max_min_composition(R, S)
print(f"{result}")

=== Creating Fuzzy Sets ===
FuzzySet(A): {np.str_('a1'): np.float64(0.6), np.str_('a2'): np.float64(0.8), np.str_('a3'): np.float64(0.5)}
FuzzySet(B): {np.str_('b1'): np.float64(0.7), np.str_('b2'): np.float64(0.9)}
FuzzySet(C): {np.str_('c1'): np.float64(0.5), np.str_('c2'): np.float64(0.8), np.str_('c3'): np.float64(0.6)}

=== Cartesian Products ===
A × B:
     b1   b2
a1  0.6  0.6
a2  0.7  0.8
a3  0.5  0.5

B × C:
     c1   c2   c3
b1  0.5  0.7  0.6
b2  0.5  0.8  0.6

=== Max-Min Composition ===
A × B∘B × C:
     c1   c2   c3
a1  0.5  0.6  0.6
a2  0.5  0.8  0.6
a3  0.5  0.5  0.5
